In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [3]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/tmp/,tmp/,0
dbfs:/user/,user/,0


In [4]:
%fs ls /FileStore/tables/sales_info.csv

path,name,size
dbfs:/FileStore/tables/sales_info.csv,sales_info.csv,196


In [5]:
path = '/FileStore/tables/sales_info.csv'

In [6]:
df = spark.read.csv(path,inferSchema=True,header=True)

In [7]:
df.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

In [8]:
df.show(5)

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
+-------+-------+-----+
only showing top 5 rows

In [9]:
df.describe().show()

+-------+-------+-------+------------------+
summary|Company| Person| Sales|
+-------+-------+-------+------------------+
 count| 12| 12| 12|
 mean| null| null| 360.5833333333333|
 stddev| null| null|250.08742410799007|
 min| APPL| Chris| 120.0|
 max| MSFT|Vanessa| 870.0|
+-------+-------+-------+------------------+

In [10]:
df.groupBy('company').mean().show()

+-------+-----------------+
company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

In [11]:
df.groupBy('company').count().show()

+-------+-----+
company|count|
+-------+-----+
 APPL| 4|
 GOOG| 3|
 FB| 2|
 MSFT| 3|
+-------+-----+

In [12]:
df.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [13]:
df.agg({'Sales':'sum'}).show()

+----------+
sum(Sales)|
+----------+
 4327.0|
+----------+

In [14]:
df.agg({'Sales':'max'}).show()

+----------+
max(Sales)|
+----------+
 870.0|
+----------+

In [15]:
df.groupBy('Company').max().show()

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

In [16]:
# imporing function
from pyspark.sql.functions import countDistinct,avg,stddev

In [17]:
df.select(countDistinct('Sales')).show()

+---------------------+
count(DISTINCT Sales)|
+---------------------+
 11|
+---------------------+

In [18]:
df.select(stddev('Sales')).show()

+------------------+
stddev_samp(Sales)|
+------------------+
250.08742410799007|
+------------------+

In [19]:
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
 Average Sales|
+-----------------+
360.5833333333333|
+-----------------+

In [20]:
df.select(stddev('Sales')).show()

+------------------+
stddev_samp(Sales)|
+------------------+
250.08742410799007|
+------------------+

In [21]:
std_sales = df.select(stddev('Sales').alias('std'))

In [22]:
std_sales.show()

+------------------+
 std|
+------------------+
250.08742410799007|
+------------------+

In [23]:
from pyspark.sql.functions import format_number

In [24]:
std_sales.select(format_number('std',2).alias('std')).show()

+------+
 std|
+------+
250.09|
+------+

In [25]:
df.orderBy('Sales').show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [26]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+